In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyomo.environ import *
from random import shuffle
%run plotting.ipynb  
np.random.seed(0)

In [2]:
def create_model(data):
    
    # unpacking data
    nrakes = len(data['a'])
    ndest = data['pendency'].shape[0]
    nterm = data['pendency'].shape[1]
    BigM = data['BigM']
    BigM2 = data['BigM2']
    
    M = ConcreteModel()
    
    # Adding variables
    M.Sa = Var(RangeSet(nrakes),RangeSet(len(data['sys'])),domain=NonNegativeReals)
    M.Sb = Var(RangeSet(nrakes),RangeSet(len(data['sys'])),domain=NonNegativeReals) 
    M.xa = Var(RangeSet(int(nrakes*(nrakes-1)/2)),RangeSet(len(data['sys'])),domain=Binary)
    M.xb = Var(RangeSet(int(nrakes*(nrakes-1)/2)),RangeSet(len(data['sys'])),domain=Binary)  
    M.w1a = Var(RangeSet(nrakes),RangeSet(data['sys'][0]),domain=Binary)
    M.w2a = Var(RangeSet(nrakes),RangeSet(data['sys'][1]),domain=Binary)
    M.w4a = Var(RangeSet(nrakes),RangeSet(sum(data['sys'][3])),domain=Binary)
    M.w1b = Var(RangeSet(nrakes),RangeSet(data['sys'][0]),domain=Binary)
    M.w2b = Var(RangeSet(nrakes),RangeSet(data['sys'][1]),domain=Binary)
    M.y = Var(RangeSet(nrakes),RangeSet(nterm),domain=Binary)
    M.v = Var(RangeSet(nrakes),RangeSet(ndest),domain=Binary)
    M.g = Var(RangeSet(nrakes),RangeSet(nterm),domain=NonNegativeIntegers,bounds=(0,data['R'][1]))
    M.n = Var(RangeSet(nrakes),RangeSet(ndest),RangeSet(nterm),domain=NonNegativeIntegers,bounds=(0,data['R'][1]))
    M.t4 = Var(RangeSet(nrakes),domain=NonNegativeReals)
    M.z = Var(RangeSet(nrakes),domain=Binary)
    
    # Adding constraints
    
    M.seqa_cons = ConstraintList() # 1
    for i in range(1,nrakes+1):
        M.seqa_cons.add(expr=M.Sa[i,1]+data['t1']<=M.Sa[i,2])
        M.seqa_cons.add(expr=M.Sa[i,2]+data['t2']<=M.Sa[i,3])
        M.seqa_cons.add(expr=M.Sa[i,3]+data['t3']<=M.Sa[i,4])
    
    M.seqb_cons = ConstraintList() # 2,3,4
    for i in range(1,nrakes+1):
        M.seqb_cons.add(expr=M.Sb[i,4] >= M.Sa[i,4]+M.t4[i])
        M.seqb_cons.add(expr=M.Sb[i,3] >= M.Sb[i,4])
        M.seqb_cons.add(expr=M.Sb[i,2] >= M.Sb[i,3]+data['t3'])
        M.seqb_cons.add(expr=M.Sb[i,1] >= M.Sb[i,2]+data['t2'])
            
    M.scheda_cons = ConstraintList() # 5,6
    idx = 1
    for i in range(1,nrakes):
        for i1 in range(i+1,nrakes+1):
            for m in range(1,data['sys'][0]+1):
                M.scheda_cons.add(expr=M.Sa[i,1] >= M.Sa[i1,2]-BigM*(1-M.xa[idx,1])-BigM*(2-M.w1a[i,m]-M.w1a[i1,m]))
                M.scheda_cons.add(expr=M.Sa[i1,1] >= M.Sa[i,2]-BigM*M.xa[idx,1]-BigM*(2-M.w1a[i,m]-M.w1a[i1,m]))
            for m in range(1,data['sys'][1]+1):
                M.scheda_cons.add(expr=M.Sa[i,2] >= M.Sa[i1,3]-BigM*(1-M.xa[idx,2])-BigM*(2-M.w2a[i,m]-M.w2a[i1,m]))
                M.scheda_cons.add(expr=M.Sa[i1,2] >= M.Sa[i,3]-BigM*M.xa[idx,2]-BigM*(2-M.w2a[i,m]-M.w2a[i1,m]))
            for m in range(1,data['sys'][2]+1):
                M.scheda_cons.add(expr=M.Sa[i,3] >= M.Sa[i1,3]-BigM*(1-M.xa[idx,3])-BigM*(2-M.y[i,m]-M.y[i1,m]))
                M.scheda_cons.add(expr=M.Sa[i1,3] >= M.Sa[i,3]-BigM*M.xa[idx,3]-BigM*(2-M.y[i,m]-M.y[i1,m]))
            for m in range(1,sum(data['sys'][3])+1):
                M.scheda_cons.add(expr=M.Sa[i,4] >= M.Sb[i1,3]-BigM*(1-M.xa[idx,4])-BigM*(2-M.w4a[i,m]-M.w4a[i1,m]))
                M.scheda_cons.add(expr=M.Sa[i1,4] >= M.Sb[i,3]-BigM*M.xa[idx,4]-BigM*(2-M.w4a[i,m]-M.w4a[i1,m]))            
            idx += 1
    
    M.schedb_cons = ConstraintList() # 7,8
    idx = 1
    for i in range(1,nrakes):
        for i1 in range(i+1,nrakes+1):
            for m in range(1,data['sys'][2]+1):
                M.schedb_cons.add(expr=M.Sb[i,3] >= M.Sb[i1,2]-BigM*(1-M.xb[idx,3])-BigM*(2-M.y[i,m]-M.y[i1,m]))
                M.schedb_cons.add(expr=M.Sb[i1,3] >= M.Sb[i,2]-BigM*(M.xb[idx,3])-BigM*(2-M.y[i,m]-M.y[i1,m]))
            for m in range(1,data['sys'][1]+1):
                M.schedb_cons.add(expr=M.Sb[i,2] >= M.Sb[i1,1]-BigM*(1-M.xb[idx,2])-BigM*(2-M.w2b[i,m]-M.w2b[i1,m]))
                M.schedb_cons.add(expr=M.Sb[i1,2] >= M.Sb[i,1]-BigM*(M.xb[idx,2])-BigM*(2-M.w2b[i,m]-M.w2b[i1,m]))
            for m in range(1,data['sys'][0]+1):
                M.schedb_cons.add(expr=M.Sb[i,1] >= M.Sb[i1,1]+data['t1']-BigM*(1-M.xb[idx,1])-BigM*(2-M.w1b[i,m]-M.w1b[i1,m]))
                M.schedb_cons.add(expr=M.Sb[i1,1] >= M.Sb[i,1]+data['t1']-BigM*(M.xb[idx,1])-BigM*(2-M.w1b[i,m]-M.w1b[i1,m]))
            idx += 1              
    
    
           
    M.releases_cons = ConstraintList() # 9
    for i in range(1,nrakes+1):
        M.releases_cons.add(expr=M.Sa[i,1] >= data['a'][i-1])
    
    M.sumw1a_cons = ConstraintList() # 10.1
    for i in range(1,nrakes+1):
        M.sumw1a_cons.add(expr=sum(M.w1a[i,m] for m in range(1,data['sys'][0]+1))==1)
    
    M.sumw2a_cons = ConstraintList() # 10.2
    for i in range(1,nrakes+1):
        M.sumw2a_cons.add(expr=sum(M.w2a[i,m] for m in range(1,data['sys'][1]+1))==1)
        
    M.sumw4a_cons = ConstraintList() # 10.4
    for i in range(1,nrakes+1):
        M.sumw4a_cons.add(expr=sum(M.w4a[i,m] for m in range(1,sum(data['sys'][3])+1))==1)
    
    M.sumw1b_cons = ConstraintList() # 11.1
    for i in range(1,nrakes+1):
        M.sumw1b_cons.add(expr=sum(M.w1b[i,m] for m in range(1,data['sys'][0]+1))==1)
        
    M.sumw2b_cons = ConstraintList() # 11.2
    for i in range(1,nrakes+1):
        M.sumw2b_cons.add(expr=sum(M.w2b[i,m] for m in range(1,data['sys'][1]+1))==1)

    M.destlineassign_cons = ConstraintList() # 12
    for i in range(1,nrakes+1):
        for d in range(1,ndest+1):
            for m in range(1,data['sys'][0]+1):
                M.destlineassign_cons.add(expr=M.w1b[i,m] >= M.v[i,d]*data['l'][d-1,m-1])
                
#     M.lineproctime_cons = ConstraintList() # 13
#     for i in range(1,nrakes+1):
#         M.lineproctime_cons.add(expr=M.t4[i] == sum(sum(M.n[i,d,t] for d in range(1,ndest+1))for t in range(1,nterm+1))/data['q']+data['tau'])
    
    M.sumv_cons = ConstraintList() # 14
    for i in range(1,nrakes+1):
        M.sumv_cons.add(expr=sum(M.v[i,d] for d in range(1,ndest+1))==1)
    
    M.sumy_cons = ConstraintList() # 15
    for i in range(1,nrakes+1):
        M.sumy_cons.add(expr=sum(M.y[i,t] for t in range(1,nterm+1))==1)
        
    M.BPCdest_cons = ConstraintList() # 16
    for i in range(1,nrakes+1):
        for d in range(1,ndest+1):
            M.BPCdest_cons.add(expr=M.v[i,d] >= data['b'][i-1,d-1])
    
    M.minmaxload_cons = ConstraintList() # 17
    for i in range(1,nrakes+1):
        M.minmaxload_cons.add(expr=data['R'][0] <= sum(sum(M.n[i,d,t] for t in range(1,nterm+1))for d in range(1,ndest+1)))
        M.minmaxload_cons.add(expr=data['R'][1] >= sum(sum(M.n[i,d,t] for t in range(1,nterm+1))for d in range(1,ndest+1)))

    M.load_cons = ConstraintList() # 18
    for i in range(1,nrakes+1):
        for d in range(1,ndest+1):
            for t in range(1,nterm+1):
                M.load_cons.add(expr=M.n[i,d,t] <= M.v[i,d]*data['pendency'][d-1,t-1])
    
    M.ittcompute_cons = ConstraintList() # 19,20(corrected), extra1
    for i in range(1,nrakes+1):
        for t in range(1,nterm+1):
            for d in range(1,ndest+1):
                M.ittcompute_cons.add(expr=M.g[i,t] <= M.n[i,d,t]+BigM2*(1-M.y[i,t])+BigM2*(1-M.v[i,d]))
                M.ittcompute_cons.add(expr=M.g[i,t] >= M.n[i,d,t]-BigM2*(1-M.y[i,t])-BigM2*(1-M.v[i,d]))
            M.ittcompute_cons.add(expr=M.g[i,t] <= data['R'][1]*M.y[i,t])
            M.ittcompute_cons.add(expr=M.g[i,t] <= data['R'][1]*M.v[i,d])
    
    M.origins_cons = ConstraintList() # extra2
    for i in range(1,nrakes+1):
        k = data['origin'][i-1]
        M.origins_cons.add(expr=M.w1a[i,k]==1)
    
    M.lastlinks_cons = ConstraintList() # extra3
    idx = 1
    for m1 in range(1,data['sys'][2]):
        for m2 in range(1,data['sys'][3][m1-1]+1):
            for i in range(1,nrakes+1):
                M.lastlinks_cons.add(expr=M.w4a[i,idx]<=M.y[i,m1])
            idx += 1
    
    M.sumpendency_cons = ConstraintList() # extra 4
    for d in range(1,ndest+1):
        M.sumpendency_cons.add(expr= sum(sum(M.n[i,d,t] for t in range(1,nterm+1)) for i in range(1,nrakes+1)) <= sum(data['pendency'][d-1]))
    

    M.procTime_cons = ConstraintList() # extra, replace 13
    for i in range(1,nrakes+1):
        M.procTime_cons.add(expr=sum(M.g[i,t] for t in range(1,nterm+1)) <= data['Pure min limit']+BigM2*(1-M.z[i]))
        M.procTime_cons.add(expr=sum(M.g[i,t] for t in range(1,nterm+1)) >= data['Pure max limit']-BigM2*(M.z[i]))
        M.procTime_cons.add(expr=M.t4[i] == data['Loading times'][1]*M.z[i]+data['Loading times'][0]*(1-M.z[i]))

#     M.removethis_cons = ConstraintList()
#     for t in range(1,nterm+1):
#         M.removethis_cons.add(expr=sum(M.g[i,t] for i in range(1,nrakes+1))>=1)
    
    #M.obj = Objective(expr=data['c1']*(sum(M.Sb[i,1]-data['a'][i-1] for i in range(1,nrakes+1)))+data['c2']*(sum(sum(sum(M.n[i,d,t] for t in range(1,nterm+1))for d in range(1,ndest+1))for i in range(1,nrakes+1))-sum(sum(M.g[i,t] for i in range(1,nrakes+1))for t in range(1,nterm+1)))-data['c3']*(sum(sum(sum(M.n[i,d,t] for t in range(1,nterm+1))for d in range(1,ndest+1))for i in range(1,nrakes+1))),sense=minimize)
    #M.obj = Objective(expr=data['c1']*(sum(M.Sb[i,1]-data['a'][i-1] for i in range(1,nrakes+1)))-data['c2']*(sum(sum(M.g[i,t] for i in range(1,nrakes+1)) for t in range(1,nterm+1))),sense=minimize)
    M.obj = Objective(expr=data['c1']*(sum(M.Sb[i,1]-data['a'][i-1] for i in range(1,nrakes+1)))+data['cont cost']*sum(sum(sum(M.n[i,d,t] for i in range(1,nrakes+1)) for d in range(1,ndest+1))for t in range(1,nterm+1)),sense=maximize)
    
    return M


In [3]:
def get_resultdataframe(data,model1,fname='dummy_result'):
    nrakes = len(data['a'])
    ndest = data['pendency'].shape[0]
    nterm = data['pendency'].shape[1]
    BigM = data['BigM']
    BigM2 = data['BigM2']
    result_rows = []
    for i in range(1,nrakes+1):
        res1 = [data['a'][i-1]]+[max(0,model1.Sa[i,j].value) for j in range(1,5)]+[model1.Sb[i,4-j].value for j in range(0,4)]+[model1.t4[i].value]
        allocations = []
        l = []
        for m in range(1,data['sys'][0]+1):
            if model1.w1a[i,m].value > 0.5:
                l.append(m)
        if len(l) > 1:
            print("Error 1",l)
        else:
            allocations.append(l[0])
        l = []
        for m in range(1,data['sys'][1]+1):
            if model1.w2a[i,m].value > 0.5:
                l.append(m)
        if len(l) > 1:
            print("Error 2",l)
        else:
            allocations.append(l[0])
        l = []
        for m in range(1,sum(data['sys'][3])+1):
            if model1.w4a[i,m].value > 0.5:
                l.append(m)
        if len(l) > 1:
            print("Error 2",l)
        else:
            allocations.append(l[0])
        l = []
        for m in range(1,data['sys'][1]+1):
            if model1.w2b[i,m].value > 0.5:
                l.append(m)
        if len(l) > 1:
            print("Error 2",l)
        else:
            allocations.append(l[0])
        l = []
        for m in range(1,data['sys'][0]+1):
            if model1.w1b[i,m].value > 0.5:
                l.append(m)
        if len(l) > 1:
            print("Error 1",l)
        else:
            allocations.append(l[0])
        l = []
        for d in range(1,ndest+1):
            if model1.v[i,d].value > 0.5:
                l.append(d)
        if len(l) > 1:
            print("Error 1",l)
        else:
            allocations.append(l[0])

        res1 += allocations

        res1.append(int(sum([model1.n[i,d,1].value for d in range(1,ndest+1)])))
        res1.append(int(sum([model1.n[i,d,2].value for d in range(1,ndest+1)])))
        l = []
        for m in range(1,data['sys'][2]+1):
            if model1.y[i,m].value > 0.5:
                l.append(m)
        if len(l) > 1:
            print("Error 2",l)
        else:
            res1.append(l[0])
        for t in range(1,nterm+1):
            res1.append(int(model1.g[i,t].value))
        result_rows.append(res1)

    result_dataframe = pd.DataFrame(data=result_rows,columns=['Arr times','Sa 1','Sa 2','Sa 3','Sa 4','Sb 4','Sb 3','Sb 2','Sb 1','t4','alloca 1','alloca 2','alloca 4','allocb 2','allocb 1','dest','pend 1','pend 2','terminal allocated']+['g'+str(t) for t in range(1,nterm+1)])
    result_dataframe.to_csv('data/'+fname+'.csv')
    return result_dataframe

In [4]:
def get_specTrains(result,m):
    r = result
    ans = []
    for i in r.index:
        for j in r.columns:
            if r.loc[i,j] >= m*24.0:
                ans.append(r.loc[i,:])
                break
    ans = pd.DataFrame(data=ans,columns=r.columns)
    return ans

In [5]:
def create_rollingModel(rolling_data,data,rolling):
    
    BigM = data['BigM']*2
    BigM2 = data['BigM2']
    
    M = create_model(data)
    if not rolling:
        return M
    cont_rakes = len(rolling_data.index)
    nrakes = len(data['a'])
    M.cont_xa = Var(RangeSet(cont_rakes),RangeSet(nrakes),RangeSet(len(data['sys'])),domain=Binary)
    M.cont_xb = Var(RangeSet(cont_rakes),RangeSet(nrakes),RangeSet(len(data['sys'])),domain=Binary)
    w1a = [[0 for _ in range(cont_rakes)]for _ in range(data['sys'][0])]
    for i in range(cont_rakes):
        w1a[int(rolling_data.loc[i,'alloca 1']-1)][i] = 1
    w2a = [[0 for _ in range(cont_rakes)]for _ in range(data['sys'][1])]
    for i in range(cont_rakes):
        w2a[int(rolling_data.loc[i,'alloca 2']-1)][i] = 1
    y = [[0 for _ in range(cont_rakes)]for _ in range(data['sys'][2])]
    for i in range(cont_rakes):
        y[int(rolling_data.loc[i,'terminal allocated']-1)][i] = 1
    w4a = [[0 for _ in range(cont_rakes)]for _ in range(sum(data['sys'][3]))]
    for i in range(cont_rakes):
        w4a[int(rolling_data.loc[i,'alloca 4']-1)][i] = 1
    w1b = [[0 for _ in range(cont_rakes)]for _ in range(data['sys'][0])]
    for i in range(cont_rakes):
        w1b[int(rolling_data.loc[i,'allocb 1']-1)][i] = 1
    w2b = [[0 for _ in range(cont_rakes)]for _ in range(data['sys'][1])]
    for i in range(cont_rakes):
        w2b[int(rolling_data.loc[i,'allocb 2']-1)][i] = 1
    
    M.rolling_scheda_cons = ConstraintList()
    for i in range(1,nrakes+1):
        for i1 in range(1,cont_rakes+1):
            for m in range(1,data['sys'][0]+1):
                #M.rolling_scheda_cons.add(inequality(rolling_data.loc[i-1,'Sa 2']-BigM*(M.cont_xa[i1,i,1])-BigM*(2-M.w1a[i1,m]-w1a[m-1][i-1]),M.Sa[i1,1],rolling_data.loc[i-1,'Sa 1']+BigM*(1-M.cont_xa[i1,i,1])+BigM*(2-M.w1a[i1,m]-w1a[m-1][i-1])))
                M.rolling_scheda_cons.add(expr=rolling_data.loc[i-1,'Sa 1'] >= M.Sa[i1,1]-BigM*(1-M.cont_xa[i1,i,1])-BigM*(2-M.w1a[i1,m]-w1a[m-1][i-1]))
                M.rolling_scheda_cons.add(expr=M.Sa[i1,1] >= rolling_data.loc[i-1,'Sa 2']-BigM*(M.cont_xa[i1,i,1])-BigM*(2-M.w1a[i1,m]-w1a[m-1][i-1]))    
            for m in range(1,data['sys'][1]+1):
                #M.rolling_scheda_cons.add(inequality(rolling_data.loc[i-1,'Sa 3']-BigM*(M.cont_xa[i1,i,2])-BigM*(2-M.w2a[i1,m]-w2a[m-1][i-1]),M.Sa[i1,2],rolling_data.loc[i-1,'Sa 2']+BigM*(1-M.cont_xa[i1,i,2])+BigM*(2-M.w2a[i1,m]-w2a[m-1][i-1])))
                M.rolling_scheda_cons.add(expr=rolling_data.loc[i-1,'Sa 2'] >= M.Sa[i1,2]-BigM*(1-M.cont_xa[i1,i,2])-BigM*(2-M.w2a[i1,m]-w2a[m-1][i-1]))
                M.rolling_scheda_cons.add(expr=M.Sa[i1,2] >= rolling_data.loc[i-1,'Sa 3']-BigM*(M.cont_xa[i1,i,2])-BigM*(2-M.w2a[i1,m]-w2a[m-1][i-1]))      
            for m in range(1,data['sys'][2]+1):
                #M.rolling_scheda_cons.add(inequality(rolling_data.loc[i-1,'Sa 4']-BigM*(M.cont_xa[i1,i,3])-BigM*(2-M.y[i1,m]-y[m-1][i-1]),M.Sa[i1,3],rolling_data.loc[i-1,'Sa 3']+BigM*(1-M.cont_xa[i1,i,3])+BigM*(2-M.y[i1,m]-y[m-1][i-1])))
                M.rolling_scheda_cons.add(expr=rolling_data.loc[i-1,'Sa 3'] >= M.Sa[i1,3]-BigM*(1-M.cont_xa[i1,i,3])-BigM*(2-M.y[i1,m]-y[m-1][i-1]))
                M.rolling_scheda_cons.add(expr=M.Sa[i1,3] >= rolling_data.loc[i-1,'Sa 4']-BigM*(M.cont_xa[i1,i,3])-BigM*(2-M.y[i1,m]-y[m-1][i-1]))      
            for m in range(1,sum(data['sys'][3])+1):
                #M.rolling_scheda_cons.add(inequality(rolling_data.loc[i-1,'Sb 3']-BigM*(M.cont_xa[i1,i,4])-BigM*(2-M.w4a[i1,m]-w4a[m-1][i-1]),M.Sa[i1,4],rolling_data.loc[i-1,'Sa 4']+BigM*(1-M.cont_xa[i1,i,4])+BigM*(2-M.w4a[i1,m]-w4a[m-1][i-1])))
                M.rolling_scheda_cons.add(expr=rolling_data.loc[i-1,'Sa 4'] >= M.Sa[i1,4]-BigM*(1-M.cont_xa[i1,i,4])-BigM*(2-M.w4a[i1,m]-w4a[m-1][i-1]))
                M.rolling_scheda_cons.add(expr=M.Sa[i1,4] >= rolling_data.loc[i-1,'Sb 3']-BigM*(M.cont_xa[i1,i,4])-BigM*(2-M.w4a[i1,m]-w4a[m-1][i-1]))      
    
    M.rolling_schedb_cons = ConstraintList()
    for i in range(1,nrakes+1):
        for i1 in range(1,cont_rakes+1):
            for m in range(1,data['sys'][2]+1):
                #M.rolling_schedb_cons.add(inequality(rolling_data.loc[i-1,'Sb 2']-BigM*(M.cont_xb[i1,i,3])-BigM*(2-M.y[i1,m]-y[m-1][i1-1]),M.Sb[i1,3],rolling_data.loc[i-1,'Sb 3']+BigM*(1-M.cont_xb[i1,i,3])+BigM*(2-M.y[i1,m]-y[m-1][i1-1])))
                M.schedb_cons.add(expr=rolling_data.loc[i-1,'Sb 3'] >= M.Sb[i1,3]-BigM*(1-M.cont_xb[i1,i,3])-BigM*(2-M.y[i1,m]-y[m-1][i-1]))
                M.schedb_cons.add(expr=M.Sb[i1,3] >= rolling_data.loc[i-1,'Sb 3']-BigM*(M.cont_xb[i1,i,3])-BigM*(2-M.y[i1,m]-y[m-1][i-1]))
            for m in range(1,data['sys'][1]+1):
                #M.rolling_schedb_cons.add(inequality(rolling_data.loc[i-1,'Sb 1']-BigM*(M.cont_xb[i1,i,2])-BigM*(2-M.w2b[i1,m]-w2b[m-1][i1-1]),M.Sb[i1,2],rolling_data.loc[i-1,'Sb 2']+BigM*(1-M.cont_xb[i1,i,2])+BigM*(2-M.w2b[i1,m]-w2b[m-1][i1-1])))
                M.schedb_cons.add(expr=rolling_data.loc[i-1,'Sb 2'] >= M.Sb[i1,2]-BigM*(1-M.cont_xb[i1,i,2])-BigM*(2-M.w2b[i1,m]-w2b[m-1][i-1]))
                M.schedb_cons.add(expr=M.Sb[i1,2] >= rolling_data.loc[i-1,'Sb 2']-BigM*(M.cont_xb[i1,i,2])-BigM*(2-M.w2b[i1,m]-w2b[m-1][i-1]))
            for m in range(1,data['sys'][0]+1):
                #M.rolling_schedb_cons.add(inequality(rolling_data.loc[i-1,'Sb 1']+data['t1']-BigM*(M.cont_xb[i1,i,1])-BigM*(2-M.w1b[i1,m]-w1b[m-1][i1-1]),M.Sb[i1,1],rolling_data.loc[i-1,'Sb 1']+BigM*(1-M.cont_xb[i1,i,1])+BigM*(2-M.w1b[i1,m]-w1b[m-1][i1-1])))
                M.schedb_cons.add(expr=rolling_data.loc[i-1,'Sb 1'] >= M.Sb[i1,1]-BigM*(1-M.cont_xb[i1,i,1])-BigM*(2-M.w1b[i1,m]-w1b[m-1][i-1]))
                M.schedb_cons.add(expr= M.Sb[i1,1] >= rolling_data.loc[i-1,'Sb 1']-BigM*(M.cont_xb[i1,i,1])-BigM*(2-M.w1b[i1,m]-w1b[m-1][i-1]))
    return M

In [6]:
'''
1. 

'''

def get_data(day):
    data = {'pendency':[],  # number of terminals x destinations - only 
        'b':[],   # bpc status of rake - 1 if a rake has to go to that destination (rakes x dest)
        'q':20.0,
        'sys':[2,3,2,[3,3]],
        'a':[],
        'l':[],
        'origin':[],
        'R':[80,90],
        'Pure min limit': 89,
        'Pure max limit': 90,
        'Loading times': [5.0,10.0],
        'cont cost': 1.0,
        'tau':0.25,
        't1':0.5,
        't2':1.0,
        't3':0.5,
        'c1':-200.0,
        'c2':1.0,
        'c3':60.0,
        'BigM':200.0,
        'BigM2':100.0}
    nrakes = 10
    ndest = 6
    nterm = 2
    l = np.array([[1,0],[1,0],[1,0],[0,1],[0,1],[0,1]])
    data['a'] = [0.0 for _ in range(nrakes)]
    t = 0.0
    for i in range(len(data['a'])):
        t += np.random.uniform(low=0.5,high=4.0)
        data['a'][i] = t+day*24.0
    bpc = np.array([[0.0 for d in range(ndest)]for i in range(nrakes)])
    bpc[4,4] = 1.0
    origin = [2, 1, 2, 2, 1, 2, 1, 1, 1, 2]
    data['b'] = bpc
    data['origin'] = origin
    data['l'] = l

    p = pd.read_csv('data/d.csv',header=None).to_numpy()
    data['pendency'] = p
    return data

In [7]:
def solveModel(M_rolling,data,fname):
    opt = SolverFactory('cplex')
    opt.options['timelimit'] = 1800
    result_rolling = opt.solve(M_rolling,tee=True)
    print("Solver status :",result_rolling.solver.status)
    print("Termination condition :",result_rolling.solver.termination_condition)
    rolling_result_dataframe = get_resultdataframe(data,M_rolling,fname=fname)
    return rolling_result_dataframe

In [8]:
ndays = 3
results_data = []
datas = []
np.random.seed(0)
for d in range(ndays):
    print("###################################")
    print("Day -",d+1)
    data = get_data(d)
    data['pendency'] += np.random.randint(low=0,high=5,size=(6,2))*50
    datas.append(data)
    if d == 0:
        M = create_rollingModel(None,data,rolling=False)
        results_data.append(solveModel(M,data,'dummy_result_day'+str(d+1)))
    else:
        rolling_data = get_specTrains(results_data[-1],d+1)
        M = create_rollingModel(rolling_data,data,rolling=True)
        results_data.append(solveModel(M,data,'dummy_result_day'+str(d+1)))
    print("###################################")

###################################
Day - 1

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 20.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2020.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\91824\AppData\Local\Temp\tmp7dtwtfz7.cplex.log' open.
CPLEX> New value for time limit in seconds: 1800
CPLEX> Problem 'C:\Users\91824\AppData\Local\Temp\tmpc9g2slwx.pyomo.lp' read.
Read time = 0.00 sec. (0.28 ticks)
CPLEX> Problem name         : C:\Users\91824\AppData\Local\Temp\tmpc9g2slwx.pyomo.lp
Objective sense      : Maximize
Variables            :     796  [Nneg: 91,  Binary: 565,  General Integer: 140]
Objective nonzeros   :     131
Linear constraints   :    2627  [Less: 2396,  Greater: 140,  Equal: 91]
  Nonzeros           :   11431


*    11+    4                       -22405.1096   -20966.6667             6.42%
*    14+    2                       -22387.9017   -20966.6667             6.35%
*   111+    6                       -22228.8426   -20966.6667             5.68%
*   134+    6                       -22100.0000   -20966.6667             5.13%
*   143+   43                       -21513.9114   -20966.6667             2.54%
*   187    78      integral     0   -21382.8273   -20966.6667     1120    1.95%
*   335   147      integral     0   -21100.0000   -20966.6667     1612    0.63%

Implied bound cuts applied:  3
Flow cuts applied:  1
Mixed integer rounding cuts applied:  15
Lift and project cuts applied:  2
Gomory fractional cuts applied:  4

Root node processing (before b&c):
  Real time             =    0.34 sec. (245.42 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.23 sec. (120.68 ticks)
  Sync time (average)   =    0.07 sec.
  Wait time (average)   =    0.00 sec.
                          ---

In [9]:
%run plotting.ipynb  
for i in range(ndays):
    plot_gantt(results_data[i],datas[i],i+1)

In [10]:
rolling_data

,Arr times,Sa 1,Sa 2,Sa 3,Sa 4,Sb 4,Sb 3,Sb 2,Sb 1,t4,...,alloca 2,alloca 4,allocb 2,allocb 1,dest,pend 1,pend 2,terminal allocated,g1,g2
0,27.297055,27.297055,27.797055,28.797055,29.297055,34.297055,34.297055,34.797055,35.797055,5.0,...,2.0,1.0,2.0,2.0,6.0,90.0,0.0,1.0,90.0,0.0
1,29.412233,29.412233,29.912233,30.912233,31.412233,36.412233,36.412233,36.912233,37.912233,5.0,...,1.0,2.0,2.0,2.0,6.0,90.0,0.0,1.0,90.0,0.0
2,32.644085,32.644085,33.144085,34.144085,34.644085,39.644085,39.644085,40.144085,41.144085,5.0,...,2.0,5.0,2.0,2.0,6.0,0.0,90.0,2.0,0.0,90.0
3,33.558045,33.558045,34.058045,35.058045,35.558045,45.558045,45.558045,46.058045,47.058045,10.0,...,3.0,6.0,2.0,1.0,3.0,90.0,0.0,2.0,0.0,0.0
4,36.297769,36.297769,36.797769,37.797769,38.297769,48.297769,48.297769,48.797769,49.797769,10.0,...,1.0,4.0,1.0,2.0,5.0,90.0,0.0,2.0,0.0,0.0
5,37.299505,37.299505,37.799505,38.799505,39.299505,44.299505,44.299505,44.799505,45.799505,5.0,...,1.0,1.0,2.0,2.0,6.0,90.0,0.0,1.0,90.0,0.0
6,41.105847,41.105847,41.605847,42.605847,43.105847,53.105847,53.105847,53.605847,54.605847,10.0,...,1.0,5.0,2.0,2.0,5.0,0.0,90.0,1.0,0.0,0.0
7,43.432316,43.432316,43.932316,44.932316,45.432316,50.432316,50.432316,50.932316,51.932316,5.0,...,1.0,3.0,2.0,2.0,6.0,90.0,0.0,1.0,90.0,0.0
8,45.383633,45.383633,45.883633,46.883633,47.383633,57.383633,57.383633,57.883633,58.883633,10.0,...,1.0,6.0,1.0,1.0,1.0,90.0,0.0,2.0,0.0,0.0
9,46.809577,46.809577,47.309577,48.309577,48.809577,58.809577,58.809577,59.309577,60.309577,10.0,...,1.0,4.0,2.0,2.0,5.0,90.0,0.0,2.0,0.0,0.0
